In [119]:
import pandas as pd
import re

In [120]:
mapping = {
    "proxy": "доверенность",
    "contract": "договор",
    "act": "акт",
    "application": "заявление",
    "order": "приказ",
    "invoice": "счет",
    "bill": "приложение",
    "arrangement": "соглашение",
    "contract offer": "договор оферты",
    "statute": "устав",
    "determination": "решение",
}

In [121]:
sample = pd.read_csv('../data/sample.csv')
sample

,class,text
0,arrangement,СОГЛАШЕНИЕ N 8\nо расторжении трудового догово...
1,arrangement,Соглашение о предоставлении опциона на заключе...
2,arrangement,Соглашение\nо реструктуризации задолженности\n...
3,arrangement,Дополнительное соглашение\r\nк договору купли-...
4,arrangement,Соглашение\nо расторжении договора об оказании...
...,...,...
496,bill,Счет № 5 от 01 октября 2020 г.\r\n\r\nПоставщи...
497,bill,Счет на оплату № от 14 октября 2020 года\r\n\r...
498,bill,Счет №23 от 12.09.2024 г.\t\t...
499,bill,"""Огурец!"" (ИП Микрюков В.В.)\t\t\t\t\t\t\r\n\t..."


In [124]:
value_counts = sample['class'].value_counts()
value_counts

class
proxy             71
contract          70
act               69
application       61
order             50
invoice           43
bill              41
arrangement       40
contract offer    25
statute           21
determination     10
Name: count, dtype: int64

In [125]:
from natasha import (
    MorphVocab,
    Doc,
    Segmenter,
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NamesExtractor,
    DatesExtractor,
    AddrExtractor,
    NewsNERTagger,
)

In [126]:
# Инициализация необходимых компонентов
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)
names_extractor = NamesExtractor(morph_vocab)
dates_extractor = DatesExtractor(morph_vocab)
addr_extractor = AddrExtractor(morph_vocab)

In [127]:
def remove_entities(text: str) -> str:

    # Создание объекта Doc для обработки текста
    doc = Doc(text)

    # Анализ текста и извлечение сущностей
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    doc.parse_syntax(syntax_parser)
    doc.tag_ner(ner_tagger)

    # Получение всех предлогов в тексте
    # prepositions = [token.text for token in doc.tokens if token.pos == 'ADP']



    for span in doc.spans:
        span.normalize(morph_vocab)
        span.extract_fact(names_extractor)
        span.extract_fact(dates_extractor)
        span.extract_fact(addr_extractor)



    # Удаление всех сущностей из текста
    for span in doc.spans:
        text = text.replace(span.text, '')
    
    # # Удаление всех предлогов из текста
    # for token in doc.tokens:
    #     if token.text in prepositions:
    #         text = text.replace(token.text, '')

    return text

In [128]:
def nat_lemma(text: str) -> str:
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    doc.parse_syntax(syntax_parser)
    for token in doc.tokens:
        token.lemmatize(morph_vocab) 

    return ' '.join([_.lemma for _ in doc.tokens])

In [129]:
import nltk
from nltk import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from collections import Counter

In [130]:
# Загрузка ресурсов
nltk.download('punkt')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /home/user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/user/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [131]:
def lemmat(text):
    if not isinstance(text, str):
        return
    # Токенизация текста
    tokens = word_tokenize(text)
    # Лемматизация каждого токена
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    # Склеивание лемматизированных токенов обратно в текст
    lemmatized_text = ' '.join(lemmatized_tokens)
    return lemmatized_text


In [132]:

replace_words = {
    "доверенность" : re.compile(r"\bд\s*о\s*в\s*е\s*р\s*е\s*н\s*н\s*о\s*с\s*т\s*ь\b"),
    "договор" : re.compile(r"\bд\s*о\s*г\s*о\s*в\s*о\s*р\b"),
    "акт": re.compile(r"\bа\s*к\s*т\b"),
    "заявление": re.compile(r"\bз\s*а\s*я\s*в\s*л\s*е\s*н\s*и\s*е\b"),
    "приказ": re.compile(r"\bп\s*р\s*и\s*к\s*а\s*з\b"),
    "счет": re.compile(r"\bс\s*ч\s*е\s*т\b"),
    "приложение": re.compile(r"\bп\s*р\s*и\s*л\s*о\s*ж\s*е\s*н\s*и\s*e\b"),
    "соглашение": re.compile(r"\bс\s*о\s*г\s*л\s*а\s*ш\s*е\s*н\s*и\s*e\b"),
    "договор оферты": re.compile(r"\bд\s*о\s*г\s*о\s*в\s*о\s*р\s*о\s*ф\s*е\s*р\s*т\s*ы\b"),
    "устав": re.compile(r"\bу\s*с\s*т\s*а\s*в\b"),
    "решение": re.compile(r"\bр\s*е\s*ш\s*е\s*н\s*и\s*е\b")
}


def clean_text(text: str) -> str:    
    for key, value in replace_words.items():
        text = re.sub(value, key, text)

    # text = re.sub(r'[^;:\?!,\.\-а-яА-Яa-zA-Z\s<]', '', text)
    text = re.sub(r'[^а-яА-Яa-zA-Z\s<]', '', text)
    text = re.sub(r'\.\.+', '', text)
    text = re.sub(r'\s+', ' ', text)
    
    text = ' '.join([word for word in text.split() if len(word) > 2])
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s]', '', text)
    return text


def data_processing(sample : pd.DataFrame) -> pd.DataFrame:
    sample["text"] = sample["text"].fillna("NaN")
    sample['text'] = sample['text'].apply(remove_entities)
    sample['text'] = sample['text'].str.lower()
    sample['text'] = sample['text'].apply(clean_text)
    return sample

In [133]:
def count_word(data:pd.DataFrame) -> None:
    text = ' '.join(data['text'])

        
    # Токенизируем текст на слова
    words = word_tokenize(text)

    # Считаем частоту каждого слова
    word_freq = Counter(words)

    # Выводим наиболее часто встречаемые слова
    most_common_words = word_freq.most_common(15)
    most_common_words = [( word, freq) for word, freq in  most_common_words if len(word)>2 ]
    print("Наиболее часто встречаемые слова:")
    for word, freq in most_common_words:
        # if len(word)>2:
            print(f"{word }: {freq} раз")
    print('----------------------------------------------------')

In [134]:
# sample = pd.read_csv('../data/sample.csv')
# sample = data_processing(sample)

# for key, value in mapping.items():
#     data_t= sample[sample['class'] == key]
#     data_t['text'] = data_t['text'].apply(lemmat)
#     count_word(data_t)

In [135]:
sample = pd.read_csv('../data/sample.csv')
sample = data_processing(sample)
sample

,class,text
0,arrangement,соглашение расторжении трудового договора октя...
1,arrangement,соглашение предоставлении опциона заключение д...
2,arrangement,соглашение реструктуризации задолженности дека...
3,arrangement,дополнительное соглашение договору куплипродаж...
4,arrangement,соглашение расторжении договора оказании услуг...
...,...,...
496,bill,счет октября поставщик покупатель васильева на...
497,bill,счет оплату октября года поставщик инн адрес п...
498,bill,счет поставщик инн кпп пгт новая покупатель ин...
499,bill,огурец адрес образец заполнения платежного пор...


In [137]:
sample['text']=sample['text'].apply(nat_lemma)
sample.to_csv('../data/lemma_sample.csv')

In [65]:
ext_with_entities = "директор бальмонт владимир павлович паспорт адрес регистрация : ул . красин , кв . номер лицевой счет : номер сотовый телефон : ---- заявление управлять компания являться собственник квартира адрес ул . красин , кв . дом находиться под управление ооо счастие быть . связь приобретение квартира г . , просить открыть лицевой счет для оплата коммунальный услуга имя . приложение : копия договор купля-продажа . выписка реестр . свидетельство право собственность . октябрь бальмонт"
cleaned_text = remove_entities(ext_with_entities)
print(cleaned_text)

директор бальмонт владимир павлович паспорт адрес регистрация : ул . красин , кв . номер лицевой счет : номер сотовый телефон : ---- заявление управлять компания являться собственник квартира адрес ул . красин , кв . дом находиться под управление ооо счастие быть . связь приобретение квартира г . , просить открыть лицевой счет для оплата коммунальный услуга имя . приложение : копия договор купля-продажа . выписка реестр . свидетельство право собственность . октябрь бальмонт


Тестирование на расширенных данных

In [82]:
sample = pd.read_csv('../data/external_with_samplelabel.csv')
sample = sample[['class','text']]
sample

,class,text
0,proxy,Доверенность\n\n\n\nг. Ижевск\n\nДата выдачи: ...
1,proxy,"ООО ""Артемида""\n\n426000, г. Ижевск, ул. Сосно..."
2,proxy,"Общество с ограниченной ответственностью ""Сигм..."
3,proxy,Доверенность\n\n\n\n\n\n\n\nг. Ижевск\n\nДвадц...
4,proxy,Доверенность\n\n\n\nг. Ижевск\n\n\n\nтринадцат...
...,...,...
1458,determination,ООО «Улыбка»\n\n\n\nРешение №6\n\n\n\nЕдинстве...
1459,determination,Решение №33\n\nЕдинственного участника ООО «Зе...
1460,determination,РЕШЕНИЕ № 123\n\nвнеочередного Общего собрани...
1461,determination,РЕШЕНИЕ № 07\n\nединственного Участника Общест...


In [83]:
sample = data_processing(sample)


In [84]:
sample['text']=sample['text'].apply(nat_lemma)
sample.to_csv('../data/lemma_external_with_samplelabel.csv')

In [86]:
sample_new = pd.read_csv('../data/external_with_samplelabel.csv')
sample_new['text'] = sample['text']

In [87]:
sample_new

,class,link,path,text,clear_text,clear_text_in_sample
0,proxy,https://assistentus.ru/wp-content/uploads/file...,obrazec-doverennost-na-registracziyu-ts-v-gibd...,доверенность дата выдача третий сентябрь два т...,доверенность г. ижевск дата выдачи: третье сен...,1
1,proxy,https://assistentus.ru/wp-content/uploads/file...,obrazec-doverennost-na-zaverenie-kopij-dokumentov,тело огрн инн кпп доверенность город двадцать ...,"ооо артемида г. ижевск, ул. сосновая, д. а, те...",1
2,proxy,https://assistentus.ru/wp-content/uploads/file...,obrazec-doverennost-na-poluchenie-denezhnyh-sr...,общество ограничить ответственность партизанск...,общество ограниченной ответственностью сигма г...,1
3,proxy,https://assistentus.ru/wp-content/uploads/file...,obrazec-doverennost-na-predstavlenie-interesov...,доверенность двадцать пятый сентябрь два тысяч...,доверенность г. ижевск двадцать пятое сентября...,1
4,proxy,https://assistentus.ru/wp-content/uploads/file...,obrazec-doverennost-na-poluchenie-trudovoj-kni...,доверенность тринадцатый март два тысяча девят...,доверенность г. ижевск тринадцатое марта две т...,1
...,...,...,...,...,...,...
1458,determination,https://assistentus.ru/wp-content/uploads/file...,obrazec-reshenie-o-smene-yuridicheskogo-adresa...,решение единственный участник смена юридически...,ооо улыбка решение единственного участника сме...,1
1459,determination,https://assistentus.ru/wp-content/uploads/file...,obrazec-reshenie-uchreditelya-o-smene-direktor...,решение единственный участник март год единств...,решение единственного участника ооо зеленая ми...,1
1460,determination,https://assistentus.ru/wp-content/uploads/file...,obrazec-resheniye-ychrediteley-ob-odobrenii-kr...,решение внеочередной место нахождение советски...,решение внеочередного общего собрания участник...,1
1461,determination,https://assistentus.ru/wp-content/uploads/file...,obrazec-reshenie-o-smene-naimenovaniya-ooo,единственный участник ограничить ответственнос...,решение единственного участника общества огран...,1


In [88]:
sample_new.to_csv('../data/lemma_external_with_samplelabel.csv')